Docs:
[1] https://userguide.mdanalysis.org/stable/examples/analysis/trajectory_similarity/clustering_ensemble_similarity.html

# **1) Initiall instructions**

In [ ]:
!pip install  -q py3dmol

In [ ]:
import os
import py3Dmol

import MDAnalysis as mda
from MDAnalysis.analysis import encore
from MDAnalysis.analysis.encore.clustering import ClusteringMethod as clm

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
"""Paste the path to the directory with the files that will be analyzed (without quotes) """
working_path = input('The path: \n ')

In [ ]:
"""How many clusters do you want? Paste number into window below"""
clusters_no = int(input("Number of clusters: "))

In [ ]:
os.chdir(working_path)
%pwd

In [ ]:
for file in os.listdir(os.getcwd()):
    if file == "step5_input.pdb":
        topology_file = file
    elif file == "step7_production.dcd":
        trajectory_file = file

u = mda.Universe(topology_file, trajectory_file)

# **2) Display Molecule**

In [ ]:
with open(topology_file, 'r') as file:
    pdb_data = file.read()

view = py3Dmol.view(width=640, height=800)
view.addModel(pdb_data, format='pdb')
view.setStyle({'cartoon':{'color':'plum'}})   #protein
view.setStyle({'within': {'distance': 5, 'sel': {'resn': 'LIG'}}}, {'stick': {'colorscheme': 'yellow'}})   #aa within 5 A from ligand
view.setStyle({'resn': 'LIG'}, {'stick': {'colorscheme': 'cyanCarbon'}})   #ligand
view.zoomTo()
view.show()

# **3) Clusterization**

In [ ]:
km = clm.KMeans(clusters_no)

In [ ]:
ces, details = encore.ces([u],
                         clustering_method=[km],
                         ncores=2)  #by default MDAnalysis uses 1 core for this task
clusters = details['clustering'][0]

In [ ]:
iterator1 = 0
while iterator1 < len(clusters):
    print(f'Cluster {iterator1 + 1}: {clusters.clusters[iterator1]}')
    iterator1 += 1

dictionary = {}
iterator2 = 1
for idx, elems in enumerate(clusters.clusters):
    dictionary.update({f"cluster_{iterator2}": len(clusters.clusters[idx].elements)})
    iterator2 += 1


same_members = True 
(key, value) = max(dictionary.items(), key=lambda x: x[1])
values_list = list(dictionary.values())
for idx in range(len(values_list) -1):
    if values_list[idx] != values_list[idx + 1]:
        same_members = False
    else:
        same_members = True

print()
if same_members is True:
    print(f'Clusters have the same number of conformations ({value} members)')
else:
    print(f'Most populated cluster: {key} ({value} members)')

In [ ]:
output_dir = os.getcwd()
os.makedirs(output_dir, exist_ok=True)

for idx, cluster in enumerate(clusters, start=1):
    cluster_atoms = u.select_atoms('all')
    cluster_filename = os.path.join(output_dir, f'cluster_{idx}.pdb')

    with mda.Writer(cluster_filename) as w:
        for frame in cluster:
            cluster_atoms.positions = u.trajectory[frame].positions
            w.write(cluster_atoms)
    print(f'Cluster {idx} saved at {cluster_filename}')

    centroid_idx = clusters.get_centroids()[idx - 1]
    cluster_centroid_filename = os.path.join(output_dir, f'cluster_{idx}_centroid_idx_{centroid_idx}.pdb')

    with mda.Writer(cluster_centroid_filename, n_atoms=u.atoms.n_atoms) as w2:
        u.trajectory[centroid_idx]
        w2.write(u)
    print(f'Centroid of Cluster {idx} saved at {cluster_centroid_filename}')    